```json
{
    "mask_name": "mask_00123.npy",
    "mask_height": 672,
    "mask_width": 630,
    "promote_type": "mask",
    "labels": {
        "3": {
            "instance_id": 3,
            "class_name": "table",
            "x1": 156,
            "y1": 333,
            "x2": 299,
            "y2": 414,
            "logit": 0.0
        },
        "5": {
            "instance_id": 5,
            "class_name": "table",
            "x1": 268,
            "y1": 463,
            "x2": 447,
            "y2": 666,
            "logit": 0.0
        }
    }
}
```

In [6]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

def analyze_npy_files(directory):
    directory = Path(directory)
    npy_files = list(directory.glob("*.npy"))
    total_nonzero_count = 0
    
    for npy_file in npy_files:
        mask = np.load(npy_file)
        
        # Get unique values and their counts
        unique_values, counts = np.unique(mask, return_counts=True)
        
        # Sum all values except 0 (corrected indexing)
        total_nonzero_count += counts[unique_values != 0].sum()
        
        # Display histogram of values with integer bins
        plt.figure()
        plt.hist(mask.flatten(), bins=np.arange(min(unique_values), max(unique_values) + 2) - 0.5, edgecolor='black')
        plt.xticks(unique_values)  # Ensure only integer ticks
        plt.title(f"Value Distribution in {npy_file.name}")
        plt.xlabel("Mask Value")
        plt.ylabel("Count")
        plt.show()
    
    print(f"Total count of all values except 0: {total_nonzero_count}")
    return total_nonzero_count

# Example usage
directory = Path("path_to_npy_files")
total_nonzero_count = analyze_npy_files(directory)

Total count of all values except 0: 0


In [ ]:
analyze_npy_files("/tmp/mask_data")

In [7]:
import numpy as np
from pathlib import Path
import cv2
import random

def convert_npy_to_png(npy_file, png_file, mode="color"):
    """
    Convert a mask .npy file to a .png image.
    Modes:
    - "grayscale": Normalize values to 0-255 range.
    - "binary": Convert nonzero values to 255.
    - "color": Assign each unique value a random color.
    """
    mask = np.load(npy_file)
    unique_values = np.unique(mask)
    
    if mode == "grayscale":
        mask_normalized = (mask / mask.max() * 255).astype(np.uint8)
        cv2.imwrite(str(png_file), mask_normalized)
    
    elif mode == "binary":
        binary_mask = np.where(mask > 0, 255, 0).astype(np.uint8)
        cv2.imwrite(str(png_file), binary_mask)
    
    elif mode == "color":
        h, w = mask.shape
        color_mask = np.zeros((h, w, 3), dtype=np.uint8)
        
        color_map = {val: [random.randint(0, 255) for _ in range(3)] for val in unique_values if val != 0}
        for val, color in color_map.items():
            color_mask[mask == val] = color
        
        cv2.imwrite(str(png_file), cv2.cvtColor(color_mask, cv2.COLOR_RGB2BGR))
    
    print(f"Saved: {png_file}")

def process_npy_directory(input_dir, output_dir, mode="color"):
    """Process all .npy files in a directory and convert them to PNG."""
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    npy_files = list(input_dir.glob("*.npy"))
    for npy_file in npy_files:
        png_file = output_dir / (npy_file.stem + ".png")
        convert_npy_to_png(npy_file, png_file, mode)

# Example usage
directory = Path("path_to_npy_files")
output_directory = Path("path_to_png_files")
process_npy_directory(directory, output_directory, mode="color")


ModuleNotFoundError: No module named 'cv2'